In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api._errors import TranscriptsDisabled, NoTranscriptFound
from langdetect import detect

from youtube_transcript_api import YouTubeTranscriptApi

In [ ]:
def guardar_archivo(df : pd.DataFrame, nombre_archivo: str) -> None:
    """
    Guarda el DataFrame en un archivo, ya sea csv, o txt.
    Args:
        df: pd.DataFrame, DataFrame a guardar
        nombre_archivo: str, nombre y ruta en donde se guardara el archivo.
    """
    df.to_csv(nombre_archivo, index=False)
    print(f"✅ Datos guardados en {nombre_archivo}")

In [3]:
def start_driver(url : str, wait_load : bool =True, delay: int = 5) -> webdriver.Chrome:
    """
    Inicializa la pagina web
    Args:
        url (str): URL de la página web a cargar.
        wait_load (bool): Si se debe esperar a que la página cargue completamente.
        delay (int): Tiempo en segundos para esperar antes de continuar.
    Returns:
        driver (webdriver.Chrome): Instancia del controlador de Chrome.
    """
    # Configuración más detallada de las opciones de Chrome
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-gpu')

    driver = webdriver.Chrome(options=chrome_options)

    # Navegar a la página para extraer videos
    driver.get(url)

    # Verificar si se debe esperar por la carga de la página
    if wait_load:
        print(f"Esperando {delay} segundos para que cargue la página...")
        time.sleep(delay)

    return driver

In [4]:
def obtener_stats(url : str, delay : int = 5) -> pd.DataFrame:
    """
    Se encarga de obtener las estadisticas con selenium.
    Se utiliza el WebDriver Manager para evitar problemas de compatibilidad con la versión de Chrome.\n
    ## Pasos:
    1. Se abre el navegador y se accede a la página de estadísticas de Sagrada.
    2. Se espera un tiempo determinado para que la página cargue completamente.
    3. Se obtienen los elementos que contienen las estadísticas.
    4. Se extraen los títulos y valores de las estadísticas.
    5. Se almacenan en un diccionario y se convierten a un DataFrame de pandas.
    6. Se cierra el navegador.
    Args:
        delay: int, tiempo de espera para que cargue la página
    Returns:
        pd.DataFrame: tabla con las estadísticas.
    """
    # Configurar Selenium con WebDriver Manager

    try:
        driver = start_driver(url)

        stats = driver.find_element(By.CLASS_NAME, "global-body-content-primary.ng-scope")

        # Obtención de los títulos de las estadísticas
        titulos = stats.find_elements(By.CLASS_NAME, "panel-title")
        titulos = [titulo for titulo in titulos if titulo.tag_name == "h3"]
        titulos_texto = [titulo.text for titulo in titulos]

        # Obtención de los valores de las estadísticas (paneles)
        paneles = stats.find_elements(By.CLASS_NAME, "panel-body")

        dic_stadisticos = {}

        for titulo, panel in zip(titulos_texto, paneles):
            if titulo not in ["RATINGS BREAKDOWN"]: # No aporta valor en la columna RATINGS BREAKDOWN
                stats_items = panel.find_elements(By.CLASS_NAME, "outline-item")
                valores = [item.text.replace("\n", " ") for item in stats_items]

                dic_stadisticos[titulo] = valores

    finally:
        driver.quit()
    return pd.DataFrame(dict([(k, pd.Series(v)) for k, v in dic_stadisticos.items()]))

In [5]:
def get_youtube_video_ids_from_bgg(url_base):
    try:
        driver = start_driver(url_base)
        # Buscar todos los enlaces a páginas de videos
        links = driver.find_elements(By.CSS_SELECTOR, 'a[href^="/video/"]')
        video_page_urls = list(set([link.get_attribute("href") for link in links]))

        print(f"Se encontraron {len(video_page_urls)} páginas de videos. Extrayendo enlaces de YouTube...")

        encabezado = [
                    "iframe",
                    "youtube_src",
                    "video_id",
                    "titulo",
                    "video_url"
                ]

        youtube_ids = []
        for video_url in video_page_urls:
            try:
                driver2 = start_driver(video_url, wait_load=False)
                iframe = driver.find_element(By.CSS_SELECTOR, 'iframe[src*="youtube.com/embed"]')
                youtube_src = iframe.get_attribute('src')
                video_id = youtube_src.split("/embed/")[-1].split("?")[0]
                titulo = driver.title
                youtube_ids.append((titulo, video_id, video_url))
                print(f"✔ {titulo} - ID: {video_id}")
            except Exception as e:
                print(f"✘ No se encontró video de YouTube en {video_url}: {e}")
            finally:
                driver2.quit()
        
        df_video = pd.DataFrame(youtube_ids, columns=encabezado)
    finally:
        driver.quit()

    return df_video

In [ ]:
def get_transcripts_from_df(df_video, output_csv_path):
    df_video["IDIOMA DETECTADO"] = None
    df_video["TRANSCRIPCIÓN"] = None

    for i, row in df_video.iterrows():
        titulo = row["titulo"]
        video_id = row["video_id"]

        try:
            transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['es', 'en'])
            text = "\n".join([seg["text"] for seg in transcript])
            detected_lang = detect(text)

            df_video.at[i, "IDIOMA DETECTADO"] = detected_lang
            df_video.at[i, "TRANSCRIPCIÓN"] = text

            print(f"✔ Transcripción guardada para video: {titulo} (Idioma: {detected_lang})")

        except (TranscriptsDisabled, NoTranscriptFound):
            print(f"✘ No se pudo obtener transcripción para video: {titulo}")
        except Exception as e:
            print(f"⚠ Error inesperado con el video {titulo}: {str(e)}")
    
    guardar_archivo(df_video, output_csv_path)
    return df_video

In [ ]:
# Estadisticas
url = "https://boardgamegeek.com/boardgame/199561/sagrada/stats"
estadisticas = obtener_stats(url)
# estadisticas['GAME RANKS'] = estadisticas['GAME RANKS'].str.replace('Historical Rank', '', regex=False)
guardar_archivo(estadisticas, "../datos/estadisticas/estadisticas.csv")
estadisticas

Esperando 5 segundos para que cargue la página...
✅ Datos guardados en ../datos/estadisticas/sagrada_stats.csv


,GAME STATS,GAME RANKS,PLAY STATS,COLLECTION STATS,PARTS EXCHANGE
0,Avg. Rating 7.472,Overall Rank 213 Historical Rank,"All Time Plays 287,557","Own 73,808",Has Parts 19
1,"No. of Ratings 44,391",Abstract Rank 10 Historical Rank,This Month 299,"Prev. Owned 4,901",Want Parts 19
2,Std. Deviation 1.16,Family Rank 45 Historical Rank,NaN,For Trade 606 Find For-Trade Matches,NaN
3,Weight 1.92 / 5,NaN,NaN,Want In Trade 832 Find Want-in-Trade Matches,NaN
4,"Comments 6,137",NaN,NaN,"Wishlist 9,323",NaN
5,"Fans 2,230",NaN,NaN,NaN,NaN
6,"Page Views 1,851,300",NaN,NaN,NaN,NaN


In [ ]:
# videos
url_videos = "https://boardgamegeek.com/boardgame/199561/sagrada/videos/all"
df_video = get_transcripts_from_df(get_youtube_video_ids_from_bgg(url_videos), "../datos/transcripciones/videos_transcripciones.csv")

Esperando 5 segundos para que cargue la página...
Se encontraron 37 páginas de videos. Extrayendo enlaces de YouTube...
✘ No se encontró video de YouTube en https://boardgamegeek.com/video/523056/sagrada/playing-sagrada-with-glass-bowls-dice-asmr-relaxat: Message: no such element: Unable to locate element: {"method":"css selector","selector":"iframe[src*="youtube.com/embed"]"}
  (Session info: chrome=135.0.7049.115); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF660B4EFA5+77893]
	GetHandleVerifier [0x00007FF660B4F000+77984]
	(No symbol) [0x00007FF6609191BA]
	(No symbol) [0x00007FF66096F16D]
	(No symbol) [0x00007FF66096F41C]
	(No symbol) [0x00007FF6609C2237]
	(No symbol) [0x00007FF66099716F]
	(No symbol) [0x00007FF6609BF07F]
	(No symbol) [0x00007FF660996F03]
	(No symbol) [0x00007FF660960328]
	(No symbol) [0x00007FF660961093]
	GetHandleVerifier [0x00